In [ ]:
import os
import glob


data_root = "/data/shared/rccars/"
mmdet_root = "../third_party/avstack-core/third_party/mmdetection"
mmseg_root = "../third_party/avstack-core/third_party/mmsegmentation"

## Dataset visualization

In [ ]:
from PIL import Image, ImageOps
import cv2
import numpy as np

sample_img_path = glob.glob(os.path.join(data_root, "oneclass-detection", "val", "images", "*.jpg"))[5]

# load image
img = Image.open(sample_img_path)
img = ImageOps.exif_transpose(img)  # do not transpose automatically
bbox = None

# add bounding box to image (x, y, w, h)
if bbox is not None:
    img = cv2.rectangle(
        np.array(img),
        (bbox[0], bbox[1]),
        (bbox[0] + bbox[2], bbox[1] + bbox[3]),
        color=(255, 0, 0),
        thickness=20,
    )
    img = Image.fromarray(img)

# scale to a manageable size
rescale = True
if rescale:
    base_width = 600
    wpercent = base_width / float(img.size[0])
    hsize = int((float(img.size[1]) * float(wpercent)))
    img = img.resize((base_width, hsize), Image.BICUBIC)

# display the image with jupyter's builtin display
display(img)

## Bounding box detection algorithm

In [1]:
from mmdet.apis import DetInferencer


# Choose to use a config
cfg = "rtmdet_m_8xb32-300e_rccars-oneclass"
# cfg = "rtmdet-ins_m_8xb32-300e_rccars-oneclass"
config = os.path.join(mmdet_root, f'configs/rccars/{cfg}.py')
checkpoint = os.path.join(mmdet_root, f'work_dirs/{cfg}/epoch_20.pth')

# Set the device to be used for evaluation
device = 'cuda:0'

# Initialize the DetInferencer
inferencer = DetInferencer(config, checkpoint, device)

NameError: name 'os' is not defined

In [ ]:
# Use the detector to do inference
sample_img_path = glob.glob(os.path.join(data_root, "oneclass-detection", "val", "images", "*.jpg"))[3]
result = inferencer(sample_img_path, out_dir='./output')

In [ ]:
Image.open(f'./output/vis/{sample_img_path.split("/")[-1]}')

## RC CAR ANNOTATION VIEWER

In [ ]:
# --- Step 1: Imports ---
import os
import json
import cv2
from matplotlib import pyplot as plt
from pycocotools.coco import COCO

# --- Step 2: Setup Paths ---
image_folder = 'images'
annotation_file = 'result.json'
output_folder = 'cropped_resized_images'
os.makedirs(output_folder, exist_ok=True)

# --- Step 3: Load COCO Annotations ---
print("Checking paths...")
print("result.json exists:", os.path.exists(annotation_file))
print("images folder exists:", os.path.exists(image_folder))

coco = COCO(annotation_file)

# --- Step 4: Load Image Info ---
image_ids = coco.getImgIds()
images = coco.loadImgs(image_ids)
print(f"Loaded {len(images)} images with annotations.")

# --- Step 5: Crop, Resize, Save, and Display ---
def crop_and_resize_image(image_info, show=True):
    file_name = os.path.basename(image_info["file_name"])
    image_path = os.path.join(image_folder, file_name)

    if not os.path.exists(image_path):
        print(f"⚠️ Image not found: {image_path}")
        return

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    ann_ids = coco.getAnnIds(imgIds=image_info['id'])
    anns = coco.loadAnns(ann_ids)

    if not anns:
        print(f"⚠️ No annotations for {file_name}")
        return

    # Compute tight crop bounds around all boxes
    x_min = min(ann['bbox'][0] for ann in anns)
    y_min = min(ann['bbox'][1] for ann in anns)
    x_max = max(ann['bbox'][0] + ann['bbox'][2] for ann in anns)
    y_max = max(ann['bbox'][1] + ann['bbox'][3] for ann in anns)

    # Optional padding (pixels)
    padding = 30
    x_min = max(int(x_min) - padding, 0)
    y_min = max(int(y_min) - padding, 0)
    x_max = min(int(x_max) + padding, image.shape[1])
    y_max = min(int(y_max) + padding, image.shape[0])

    # Crop and resize
    cropped = image[y_min:y_max, x_min:x_max]
    resized = cv2.resize(cropped, (640, 480))

    # Save image
    save_path = os.path.join(output_folder, file_name)
    cv2.imwrite(save_path, cv2.cvtColor(resized, cv2.COLOR_RGB2BGR))
    print(f"✅ Saved: {save_path}")

    # Display image with category labels (optional)
    if show:
        for ann in anns:
            x, y, w, h = ann['bbox']
            label = coco.loadCats(ann['category_id'])[0]['name']

            # Adjust box to cropped region
            x -= x_min
            y -= y_min
            scale_x = 640 / (x_max - x_min)
            scale_y = 480 / (y_max - y_min)

            x *= scale_x
            y *= scale_y
            w *= scale_x
            h *= scale_y

            cv2.rectangle(resized, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)
            cv2.putText(resized, label, (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.8, (255, 0, 0), 2, cv2.LINE_AA)

        plt.imshow(resized)
        plt.axis('off')
        plt.show()

for i, img in enumerate(images[:319]):  # change [:5] to [:] to run all
    crop_and_resize_image(img)

crop_and_resize_image(images[300])